In [74]:

##
##CARREGUEM LLIBRERIES
##

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [75]:

##
##PREPAREM VARIABLES QUE NECESSITEM:
##

#URL base de les cadires d'ikea
url= "https://www.ikea.com/es/es/cat/sillas-fu002/"

#Creem una llista buida per cada atribut que volem seleccionar
llista_nom_producte=[]
llista_tipus_subproducte=[]
llista_preu_producte=[]
llista_url_producte=[]

#Nombre de pàgines de ikea de cadires: 23
n_pag_inici=1
n_pag_final=23


##
## BUCLE: Recollim les dades per cada pàgina de cadires de la web d'ikea
##


for i in range(n_pag_inici,n_pag_final):
    
    ##
    ##CARREGAR PÀGINA
    ##
    
    #Descarregar pagina web
    page=requests.get(url)

    #Contingut en brut de la pàgina
    soup=BeautifulSoup(page.content)

    #Estructura per detectar la informació d'interès
    soup_pretty=soup.prettify()


    
    ##
    ##AGAFAR DADES
    ##
    
    #Agafem nom producte 

    for linia in soup.find_all('span', attrs={'class': 'product-compact__name'}):
        span_nom = linia.next_element
        llista_nom_producte.append(span_nom.string)


    #Agafem tipus subproducte ( el producte és "cadira")

    for linia in soup.find_all('span', attrs={'class': 'product-compact__type'}):
        span_subproducte = linia.next_element
        llista_tipus_subproducte.append(span_subproducte.string)


    #Agafem preu

    for linia in soup.find_all('span', attrs={'class': 'product-compact__price__value'}):
        span_preu = linia.next_element
        llista_preu_producte.append(span_preu.string)
        
        
    
        
    #Agafem url per accedir a cada producte
    
    for a in soup.find_all('div', attrs={'class':'product-compact__spacer'}):
        href = a.find('a').get('href')
        llista_url_producte.append(href)
    

        

    ##
    ##PASSAR A LA SEGÜENT URL
    ##

    #Donem la url de la següent pàgina, que es troba dins del codi següent de la mateixa pàgina:
    # <link href="https://www.ikea.com/es/es/cat/sillas-fu002/page-#/" rel="next"/>
    
    link=soup.find(rel="next")  #Busquem la línia que conté rel="next"
    url=link.get('href')        #Agafem el link que està a href
    

##
##MOSTREM LLISTES CREADES
##

#print("Llista amb els noms dels productes\n",llista_nom_producte) 
#print("Llista amb els tipus de subproductes\n",llista_tipus_subproducte) 
#print("Llista amb els preus dels productes\n",llista_preu_producte) 
#print ("Llista amb les url per accedir a cada producte", llista_url_producte)



In [111]:

##
##CARREGAR PÀGINA DE CADA PRODUCTE
##
    
#llista_informacio=[]       #llista que es volia crear pel codi que no funciona
llista_descripcio=[]

lenght_llista= len(llista_url_producte)


##
##BUCLE : Agafem descripció i informació de cada pàgina pròpia del producte
##     

#Fem un bucle accedint a cada producte a partir de la llista de les url,
#És a dir, accedim a  cada pàgina de cadires de la web d'ikea per recollir dades de cada producte


for i in range (0,lenght_llista):
    #Carreguem Url pàgina i el contingut
    page_producte=requests.get(llista_url_producte[i])
    soup_producte=BeautifulSoup(page_producte.content)
  
    
    #Agafem Descripció producte

    descripcio=""
    for div in soup_producte.find_all('div', attrs={'class':'product-pip__benefit-summary'}):
        descripcio= div.next_element.string
    if descripcio == "":
        llista_descripcio.append("")
    else   :
        llista_descripcio.append(descripcio) 
        
        
    #Afagem Informació producte
    #Aquest codi no funciona!! Per length_llista<=10 si que funciona.
    #info=soup_producte.find(itemprop="description").get('content')
    #llista_informacio.append(info)



##
##MOSTREM LLISTES CREADES
##


#print("Llista amb la descripció del producte", llista_descripcio)
#print("Llista amb la informació del producte", llista_informacio)



In [112]:
##
##CARREGUEM DADES A TAULA
##

nomcolumna=["nom_producte"]

#Creem dataframe amb els noms del producte
cadires=pd.DataFrame(llista_nom_producte, columns=nomcolumna )

#Afegim columna amb els tipus de subproducte
cadires.loc[:,'tipus_subproducte']=llista_tipus_subproducte

#Afegim columna amb el preu del producte
cadires.loc[:,'preu_producte']=llista_preu_producte

#Afegim columna amb la url del producte
cadires.loc[:,'url_producte']=llista_url_producte

#Afegim columna amb la descripció del producte
cadires.loc[:,'descripcio_producte']=llista_descripcio

#Afegim columna amb la informacio del producte
#cadires.loc[:,'informació_producte']=llista_informacio  #el codi per extreure la info no funciona

#Mostrem taula amb les dades de cadires
print (cadires)





                    nom_producte  \
0                        TRANARÖ   
1                       TÄNKVÄRD   
2                         BUSKBO   
3                         FANBYN   
4                     ULRIKSBERG   
5            NORRÅKER / RÖNNINGE   
6            NORRÅKER / NORRARYD   
7                         BEKVÄM   
8                       NORDMYRA   
9                      RENBERGET   
10                       DAGOTTO   
11                        BEKVÄM   
12                      TEODORES   
13                         GUNDE   
14                          AGEN   
15                        INGOLF   
16                         PELLO   
17                      TEODORES   
18                         KOLON   
19                         NISSE   
20                        MARIUS   
21                       ANTILOP   
22                       ANTILOP   
23                        STEFAN   
24                      JOKKMOKK   
25                          ADDE   
26                      EKED

In [113]:

##
##DECARREGAR DADES EXTRETES A CSV
##

filePath= "C:/Users/Laura/Desktop/UOC/Tipologia i cicle de vida de les dades/PRAC 1/cadires_ikea.csv"

cadires.to_csv(filePath, header=True, index=False, encoding='utf-8')

